# 0.**Librerias**

solo para train, pero se pueden tomar de test tambien, cambiando el path

In [2]:
# 1. Crear carpeta y entrar
!mkdir cochrane_data
%cd cochrane_data

# 2. Inicializar git vacío
!git init

# 3. Agregar remote
!git remote add -f origin https://github.com/feliperussi/bridging-the-gap-in-health-literacy.git

# 4. Activar modo sparse checkout
!git config core.sparseCheckout true

# 5. Indicar SOLO las carpetas que queremos
!echo "data_collection_and_processing/Data Sources/Cochrane/train/pls" >> .git/info/sparse-checkout
!echo "data_collection_and_processing/Data Sources/Cochrane/train/non_pls" >> .git/info/sparse-checkout

# 6. Descargar solo esa parte del repo
!git pull origin main


/content/cochrane_data
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/cochrane_data/.git/
Updating origin
remote: Enumerating objects: 72074, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72074 (delta 0), reused 2 (delta 0), pack-reused 72071 (from 2)
Receiving objects: 100% (72074/72074), 315.90 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (2991/2991), done.
From https://github.com/feliperussi/bridging-the-gap-in-health-literacy
 * [new branch]

In [3]:
import glob, pandas as pd
import os
import re

# Número máximo de pares
pares = 8000

pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/pls/*.txt"))
non_pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/non_pls/*.txt"))

# === Filtrar archivos que contengan 'accumulated' en AMBOS ---
pls_files = [f for f in pls_files if "accumulated" not in os.path.basename(f).lower()]
non_pls_files = [f for f in non_pls_files if "accumulated" not in os.path.basename(f).lower()]

print(f"📚 Total PLS (sin accumulated): {len(pls_files)}")
print(f"📂 Total non-PLS (sin accumulated): {len(non_pls_files)}")

def extract_id(filename):
    base = os.path.basename(filename)
    match = re.match(r"(.+?)\.pub\d+", base)
    return match.group(1) if match else base.split(".txt")[0]

pls_dict = {extract_id(p): p for p in pls_files}
non_pls_dict = {extract_id(n): n for n in non_pls_files}

data = []
for base_id, non_path in non_pls_dict.items():
    if base_id in pls_dict:
        pls_path = pls_dict[base_id]
        with open(pls_path, "r", encoding="utf-8") as f1, open(non_path, "r", encoding="utf-8") as f2:
            data.append({
                "id": base_id,
                "file_pls": os.path.basename(pls_path),
                "file_non_pls": os.path.basename(non_path),
                "pls": f1.read().strip(),
                "non_pls": f2.read().strip()
            })
    if len(data) >= pares:
        break

df = pd.DataFrame(data)

output_path = "cochrane_pairs_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Emparejados {len(df)} pares por nombre base (sin 'accumulated' en ninguno)")
print(f"💾 Guardado en: {output_path}")
df.head(3)


📚 Total PLS (sin accumulated): 4797
📂 Total non-PLS (sin accumulated): 7251

✅ Emparejados 3426 pares por nombre base (sin 'accumulated' en ninguno)
💾 Guardado en: cochrane_pairs_clean.csv


,id,file_pls,file_non_pls,pls,non_pls
0,10.1002-14651858.CD000006,10.1002-14651858.CD000006.pub2-pls.txt,10.1002-14651858.CD000006.pub2-abstract.txt,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...
1,10.1002-14651858.CD000009,10.1002-14651858.CD000009.pub4-pls.txt,10.1002-14651858.CD000009.pub4-abstract.txt_se...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...
2,10.1002-14651858.CD000012,10.1002-14651858.CD000012.pub4-pls.txt,10.1002-14651858.CD000012.pub4-abstract.txt_se...,Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl..."


In [4]:
import re
from collections import Counter
import unicodedata

candidate_titles = Counter()

for file in pls_files:
    with open(file, encoding="utf-8") as f:
        lines = f.read().splitlines()

    for line in lines:
        s = line.strip()

        # Detectar si tiene pinta de título
        if (
            1 <= len(s.split()) <= 7          # líneas cortas
            and s[0].isupper()               # empieza en mayúscula
            and not s.endswith(".")          # títulos NO terminan en punto
            and len(s) < 80                  # para evitar párrafos cortos
        ):
            candidate_titles[s.lower()] += 1

print("➡️ Títulos detectados (sin normalizar):")
for title, count in candidate_titles.most_common(30):
    print(f"{count} → {title}")


➡️ Títulos detectados (sin normalizar):
1868 → background
1538 → study characteristics
1480 → key results
1289 → review question
1031 → quality of the evidence
314 → search date
308 → key messages
288 → conclusions
224 → what is the issue?
221 → how up‐to‐date is this review?
214 → quality of evidence
213 → what did we find?
203 → what does this mean?
199 → certainty of the evidence
199 → what did we do?
182 → what is the aim of this review?
177 → what evidence did we find?
177 → what was studied in the review?
165 → why is this important?
157 → conclusion
110 → what did we want to find out?
92 → study funding sources
88 → what we found
84 → results
83 → what are the limitations of the evidence?
78 → main results
65 → key findings
62 → how up to date is this evidence?
61 → why is this review important?
59 → key results and quality of the evidence


In [5]:
import re
import unicodedata
from collections import Counter

# ------------------------------------------------------
# 1) DETECTAR TODOS LOS TÍTULOS EN LOS PLS
# ------------------------------------------------------

candidate_titles = Counter()

for file in pls_files:   # <-- ya filtrados sin "accumulated"
    with open(file, encoding="utf-8") as f:
        lines = f.read().splitlines()

    for line in lines:
        s = line.strip()

        # Regla para detectar títulos
        if (
            1 <= len(s.split()) <= 7          # pocas palabras
            and s[0].isupper()               # empieza en mayúscula
            and not s.endswith(".")          # no acaba en punto
            and len(s) < 80                  # no demasiado largo
        ):
            candidate_titles[s.lower()] += 1

print("\n=== TÍTULOS DETECTADOS (sin normalizar) ===")
for title, count in candidate_titles.most_common(30):
    print(f"{count} → {title}")


# ------------------------------------------------------
# 2) NORMALIZAR TÍTULOS (evitar duplicados)
# ------------------------------------------------------

def normalize_title(s):
    s = s.lower().strip()
    s = s.rstrip(":")
    s = unicodedata.normalize("NFKD", s)
    s = s.replace("  ", " ")
    return s

normalized_counts = Counter()

for title, count in candidate_titles.items():
    normalized_counts[normalize_title(title)] += count

print("\n=== TÍTULOS NORMALIZADOS ===")
for t, c in normalized_counts.most_common(40):
    print(f"{c} → {t}")


# ------------------------------------------------------
# 3) PREPARAR LISTA DE TÍTULOS Y REGEX
# ------------------------------------------------------

SECTION_NAMES = list(normalized_counts.keys())

SECTION_REGEX = [
    re.compile(rf"^{re.escape(name)}\b", re.IGNORECASE)
    for name in SECTION_NAMES
]


# ------------------------------------------------------
# 4) DETECTAR PATRONES DE SECCIONES POR ARCHIVO
# ------------------------------------------------------

pattern_counts = Counter()

for file in pls_files:
    with open(file, encoding="utf-8") as f:
        lines = f.read().splitlines()

    found = []

    for line in lines:
        clean = normalize_title(line.strip())
        for name, reg in zip(SECTION_NAMES, SECTION_REGEX):
            if reg.match(clean):
                found.append(name)
                break

    if found:
        pattern_counts[tuple(found)] += 1
    else:
        pattern_counts[("NO_TITLES",)] += 1


# ------------------------------------------------------
# 5) MOSTRAR LOS PATRONES MÁS COMUNES
# ------------------------------------------------------

print("\n=== PATRONES DE SECCIONES MÁS FRECUENTES ===")
for pattern, count in pattern_counts.most_common(30):
    print(f"{count} → {pattern}")



=== TÍTULOS DETECTADOS (sin normalizar) ===
1868 → background
1538 → study characteristics
1480 → key results
1289 → review question
1031 → quality of the evidence
314 → search date
308 → key messages
288 → conclusions
224 → what is the issue?
221 → how up‐to‐date is this review?
214 → quality of evidence
213 → what did we find?
203 → what does this mean?
199 → certainty of the evidence
199 → what did we do?
182 → what is the aim of this review?
177 → what evidence did we find?
177 → what was studied in the review?
165 → why is this important?
157 → conclusion
110 → what did we want to find out?
92 → study funding sources
88 → what we found
84 → results
83 → what are the limitations of the evidence?
78 → main results
65 → key findings
62 → how up to date is this evidence?
61 → why is this review important?
59 → key results and quality of the evidence

=== TÍTULOS NORMALIZADOS ===
1874 → background
1543 → study characteristics
1485 → key results
1294 → review question
1036 → quality of

In [6]:
# ================================
# Agrupar patrones en 4 macrogrupos
# ================================

macro_groups = {
    "A_Cochrane_classic": 0,
    "B_Issue_Important_Evidence": 0,
    "C_No_titles": 0,
    "D_Others": 0
}

grouped_examples = {k: [] for k in macro_groups.keys()}  # Para guardar ejemplos

for pattern, count in pattern_counts.items():

    pattern_set = set(pattern)

    # --- Grupo C: sin títulos ---
    if pattern == ("NO_TITLES",):
        macro_groups["C_No_titles"] += count
        grouped_examples["C_No_titles"].append((pattern, count))
        continue

    # --- Grupo A: estilo Cochrane clásico ---
    if any(p in pattern_set for p in [
        "review question",
        "study characteristics",
        "search date",
        "key results",
        "quality of the evidence",
        "quality of evidence"
    ]):
        macro_groups["A_Cochrane_classic"] += count
        grouped_examples["A_Cochrane_classic"].append((pattern, count))
        continue

    # --- Grupo B: estilo narrativo ---
    if any(p in pattern_set for p in [
        "what is the issue",
        "why is this important",
        "what evidence did we find",
        "what does this mean"
    ]):
        macro_groups["B_Issue_Important_Evidence"] += count
        grouped_examples["B_Issue_Important_Evidence"].append((pattern, count))
        continue

    # --- Grupo D: otros ---
    macro_groups["D_Others"] += count
    grouped_examples["D_Others"].append((pattern, count))


print("\n=== Macro-grupos encontrados ===")
for group, count in macro_groups.items():
    print(f"{group}: {count} PLS")

print("\n=== Ejemplos por grupo ===")
for group, examples in grouped_examples.items():
    print(f"\n--- {group} ---")
    for p, c in sorted(examples, key=lambda x: -x[1])[:10]:
        print(f"{c} → {p}")



=== Macro-grupos encontrados ===
A_Cochrane_classic: 2226 PLS
B_Issue_Important_Evidence: 281 PLS
C_No_titles: 863 PLS
D_Others: 1427 PLS

=== Ejemplos por grupo ===

--- A_Cochrane_classic ---
140 → ('review question', 'background', 'study characteristics', 'key results', 'quality of the evidence')
41 → ('review question', 'background', 'study characteristics', 'key results', 'quality of evidence')
27 → ('background', 'study characteristics', 'key results', 'quality of the evidence')
26 → ('review question', 'background', 'search date', 'the evidence', 'study characteristics', 'key results', 'quality of the evidence')
24 → ('review question', 'background', 'study characteristics', 'the evidence', 'key results', 'quality of the evidence')
18 → ('review question', 'background', 'study characteristics', 'key results', 'quality of the evidence', 'the evidence')
18 → ('background', 'review question', 'study characteristics', 'key results', 'quality of the evidence')
17 → ('review question